## Objective: Narrow down the set of job postings to those that are most similar to our resume in preparation for further analysis.

### Step 1: Transform job posting text and our resume into TFIDF vectors using scikit-learn’s TFIDF vectorizer class.

In [5]:
# Group library imports in the beginning

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import os

In [61]:
# Fetch DataFrame from previous steps

jobs = pd.read_pickle('step1_df.pk')
jobs.head()

,title,body,bullets
0,"Quantitative Analyst - Boston, MA 02116","Quantitative Analyst - Boston, MA 02116\nQuant...",()
1,"Data Scientist - Mountain View, CA","Data Scientist - Mountain View, CA\nGroundTrut...","(Help senior members of the team to explore, d..."
2,"Data Scientist - Seattle, WA","Data Scientist - Seattle, WA\nA Bachelor or Ma...",(A Bachelor or Masters Degree in a highly quan...
3,Senior Natural Language Processing (NLP) Engin...,Senior Natural Language Processing (NLP) Engin...,(Join a small team creating a proprietary NLU ...
4,"FLEXO FOLDER GLUER OPER - McClellan, CA - McCl...","FLEXO FOLDER GLUER OPER - McClellan, CA - McCl...",()


In [62]:
# Convert resume into TFIDF vector
my_resume = "/Users/samanthaberk/Documents/resume-job-posting-nlp-project/data/Liveproject Resume.txt"

my_resume = [line.strip() for line in open(my_resume)]

vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit(my_resume)
resume_vector = vectorizer.transform(my_resume)
print(resume_vector.shape)
print(resume_vector.toarray())


(34, 52)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Step 2: Compute the cosine similarity between the vectorized resume and the job postings using sklearn’s cosine similarity function.

In [68]:
# Convert bullets columns to list
jobs_list = list(jobs['bullets'])

job = jobs_list[1]
vectorizer.fit(job)
job_vector = vectorizer.transform(job)
print(job_vector.shape)
print(job_vector.toarray())


(11, 67)
[[0.30343771 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.30343771 0.         0.
  0.         0.         0.         0.25936751 0.         0.
  0.         0.30343771 0.         0.         0.2280992  0.
  0.         0.2280992  0.         0.         0.         0.30343771
  0.         0.         0.         0.30343771 0.         0.
  0.         0.30343771 0.         0.         0.30343771 0.
  0.         0.         0.         0.         0.         0.
  0.30343771 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.30343771 0.
  0.        ]
 [0.         0.29074331 0.3401447  0.3401447  0.         0.
  0.         0.         0.         0.3401447  0.22850491 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.29074331 0.         0.
  0.3401447  0.         0.         0.         0.         0.
  0.     

### Step 3: Sort the job postings based on similarity to our resume, and choose an appropriate cutoff for selecting the most similar jobs. Store the most similar job postings in a new DataFrame for later use, and save the DataFrame to disk.